In [1]:
import random
random.seed(2015)
import csv
from string import *
import numpy as np


In [2]:
train_data = open('/Users/junanqu/Desktop/text_p2/train.txt','r').readlines()
test_data = open('/Users/junanqu/Desktop/text_p2/test.txt','r').readlines()

tags = ["B-ORG", "B-MISC", "B-PER", "B-LOC", "I-ORG", "I-MISC", "I-PER", "I-LOC", "O"]
transition_counts = np.zeros((9, 9))
ner_dict = {}
for k in range(9):
    ner_dict[tags[k]] = k

In [3]:
# a customized class to store training-example
class integrated_instance():
    def __init__(self):
        self.sent = ""  
        self.word = []  
        self.pos = [] 
        self.ner = [] 
        self.net_test = [] 
        self.map = {} 

In [4]:
def read_data(data, type = "train"):
    if type == "test":
        sent = ""
        for i in range(0, len(data), 3):
            sent += data[i] + "\t"
        pos = ""
        for i in range(1, len(data), 3):
            pos += data[i] + "\t"
        ner = ""
        for i in range(2, len(data), 3):
            ner += data[i] + " "
        
        translation = {"\r\n": None}
        
        sent = sent.translate(translation)
        pos = pos.translate(translation)
        ner = ner.translate(translation)

        
        test_instance = integrated_instance()
        test_instance.sent = sent[:-1]
        test_instance.word = sent.split("\t")[:-1]
        test_instance.pos = pos.split("\t")[:-1]
        test_instance.ner = ner.split(" ")[:-1]
        return test_instance
    
    list_sent_instance = []    
    
    max_idx_line = len(data)
    idx_line_loaded = 0
    while (idx_line_loaded + 3 <= max_idx_line):
        # create a sent_instance object        
        new_instance = integrated_instance()
        # break strings and         
        new_instance.sent = data[idx_line_loaded][:-1] # -2 to get rid of "\r\n"
        new_instance.word = data[idx_line_loaded][:-1].split("\t")
        new_instance.pos = data[idx_line_loaded+1][:-1].split("\t")
        new_instance.ner = data[idx_line_loaded+2][:-1].split("\t")
        
        list_sent_instance.append(new_instance)
        idx_line_loaded += 3
    return list_sent_instance

In [5]:
data = read_data(train_data)
test = read_data(test_data, type="test")

In [6]:
def emission_prob(sentence_list):
    emis_dic={'O':{},'B-LOC':{},'B-ORG':{},'B-PER':{},'B-MISC':{},'I-LOC':{},'I-ORG':{},'I-PER':{},'I-MISC':{}}
    for sentence in sentence_list:
        for i in range(0, len(sentence.ner),1):
            current_word = sentence.word[i]
            current_ner = sentence.ner[i]
            if (len(current_ner)==0):
                continue
            if current_word not in emis_dic[current_ner]:
                emis_dic[current_ner][current_word]=1.0
            else:
                emis_dic[current_ner][current_word]+=1.0
    
    for ner in emis_dic:
        num_ner = 0.0
        for word in emis_dic[ner]:
            num_ner+=emis_dic[ner][word]
        for word in emis_dic[ner]:
            emis_dic[ner][word]/=num_ner
    return emis_dic

In [7]:
def get_start_probability(sentence_list):
    num_sentence = len(sentence_list)
    start_ner_counts = {}
    for sentence in sentence_list:
        ner1 = sentence.ner[0]
        if ner1 in start_ner_counts:
            start_ner_counts[ner1] += 1.0
        else:
            start_ner_counts[ner1] = 1.0
    for ner in start_ner_counts:
        start_ner_counts[ner] /= num_sentence
    
    return start_ner_counts


In [8]:
def transition_counts(sent_list):
    tags = ["B-ORG", "B-MISC", "B-PER", "B-LOC", "I-ORG", "I-MISC", "I-PER", "I-LOC", "O"]
    transition_counts = np.zeros((9, 9))
    global ner_dict
    for k in range(9):
        ner_dict[tags[k]] = k
    for i in range(0, len(data)):
        this_tags = data[i].ner
        for j in range(1, len(this_tags)):
            if (len(this_tags[j])==0):
                continue
            op = ner_dict[this_tags[j-1]]
            cond = ner_dict[this_tags[j]]
            transition_counts[op][cond] = transition_counts[op][cond]+1
        
    return transition_counts

In [9]:
def transition_prob(transition_counts, k):
    transition_counts = np.add(transition_counts, k)
    transition_probs = np.zeros((9,9))
    for i in range(len(transition_counts)): 
        transition_probs[i] = np.divide(transition_counts[i], np.sum(transition_counts[i]))
    return transition_probs[:]

In [10]:
word_dict = {}
def emission_2D_prob(data):
    all_tokens = []
    all_tags = []

    for p in range(len(data)):

        this_sentence = data[p].word
        this_tags = data[p].ner

        if len(this_sentence) == len(this_tags):

            for q in range(len(this_sentence)):

                all_tokens.append(this_sentence[q])
                all_tags.append(this_tags[q])
    word_types = np.unique(all_tokens)
    
    global word_dict
    
    for t in range(len(word_types)):
        word_dict[word_types[t]] = t

    lexical_counts = np.zeros((len(word_types)+1, 9))

    for u in range(len(all_tokens)):
        if (len(all_tags[u])==0):
            continue
        lexical_counts[word_dict[all_tokens[u]]][ner_dict[all_tags[u]]] = lexical_counts[word_dict[all_tokens[u]]][ner_dict[all_tags[u]]] + 1
    (rows, cols) = np.shape(lexical_counts)

    for bb in range(cols):

        unk_count = 0

        this_column = lexical_counts[:,bb]

        for aa in range(rows):
            if lexical_counts[aa][bb] == 1:
                lexical_counts[aa][bb] = 0
                unk_count = unk_count + 1

        lexical_counts[rows-1][bb] = unk_count
    
#     Smoothing here
    k = 0.01
    lexical_counts = np.add(lexical_counts, k)

    lexical_probs = np.zeros((rows, cols))

    for zz in range(len(lexical_counts)):

        lexical_probs[zz] = np.divide(lexical_counts[zz], np.sum(lexical_counts[zz]))

    
    return lexical_probs

In [11]:
# the number of counts for training dataset for transition probability
transition_tr_data_counts = transition_counts(data)

# the transition probabity using K-smoothing method on transition counts
transition_prob_chart = transition_prob(transition_tr_data_counts, 0.01)

# from training data to get the probability of having each tag as the starting tag
start_tag_prob = get_start_probability(data)

# emission probability by having a 2D matrix n*m

emission_prob_smoothed = emission_2D_prob(data)

STATES = ('O','B-LOC','B-ORG','B-PER','B-MISC','I-LOC','I-ORG','I-PER','I-MISC')

for key in STATES:
    if key in start_tag_prob:
        start_tag_prob[ner_dict[key]] = start_tag_prob[key]


# Need to get the unk prob for the emission

9

In [14]:
def viterbi(s_prob, t_prob, e_prob, states, obs):
    
    #best_prob_state_ind maintains the best probability of getting to a given index ending on a given state
    #returns the prob of [index][state]
    best_prob_state_ind = dict()
    for i in range(len(obs)):
        best_prob_state_ind[i] = dict()
        for state in states:
            state = ner_dict[state]
            best_prob_state_ind[i][state] = 1
    
    #best_path_state_ind maintains the best path of getting to a given index ending on a given state
    #returns the path of [state][index]
    best_path_state_ind = dict()
    for state in states:
        state = ner_dict[state]
        best_path_state_ind[state] = dict()
    
    #initializes path probabilities using start_probabilities
    for state in states:
        state = ner_dict[state]

#         print(s_prob[state])
#         print(e_prob[word_dict[obs[0]]][state])
        
#         what does e_prob[state][obs[0]] do?
        if state in s_prob:
            best_prob_state_ind[0][state] = s_prob[state] * e_prob[word_dict[obs[0]]][state]
        else:
            best_prob_state_ind[0][state] = 1e-10 * e_prob[word_dict[obs[0]]][state]
        best_path_state_ind[state][0] = [state]
    
    #iterates through index,state and fills both dictionaries
    for i in range(1, len(obs)):
        for state in states:
            state = ner_dict[state]
            best_prob = 0
            best_path = []
            
            #iterates through possible previous states to find best probability
            for prev_state in states:
                prev_state = ner_dict[prev_state]
                prev_prob = best_prob_state_ind[i - 1][prev_state]
                transition = t_prob[prev_state][state]
                if obs[i] in word_dict:
                    emission = e_prob[word_dict[obs[i]]][state]
                else:
                    emission = e_prob[word_dict['UNK'][state]]                     
                #probability defined
                if(prev_prob * transition * emission > best_prob):
                    best_prob = prev_prob * transition * emission
                    if i-1 in best_path_state_ind[prev_state]:
                        best_path = best_path_state_ind[prev_state][i - 1].append(state)

            #update dicts
            best_prob_state_ind[i][state] = best_prob
            best_path_state_ind[prev_state][i] = best_path
    
    #find best path
    for state in states:
        best_prob = 0
        best_path = []
        
        if best_prob_ind[len(obs)][state] > best_prob:
            best_path = best_path_state_ind[state][len(obs)]
    
    return best_path

In [15]:
viterbi_tab = viterbi(start_tag_prob,transition_prob_chart,emission_prob_smoothed, STATES, test.word)

AttributeError: 'numpy.ndarray' object has no attribute 'get'